In [ ]:
!pip install pyspark psycopg2 sqlalchemy python-dotenv

In [57]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrameWriter
from pyspark.sql.functions import monotonically_increasing_id
import psycopg2
import os
from dotenv import load_dotenv
from pyspark.sql.functions import expr


In [26]:
load_dotenv()

True

In [29]:
password = os.getenv('pg_password')

In [ ]:
!pip install jpype1


In [31]:
spark =  SparkSession.builder \
                     .appName('Alien Bank')\
                     .config('spark.jars', 'postgresql-42.7.3.jar')\
                     .getOrCreate()

In [32]:
spark

In [33]:
# Extracting raw data into spark Dataframe
df = spark.read.csv(r'data/alien_bank_transactions.csv', header=True, inferSchema=True)

24/06/29 11:39:40 INFO InMemoryFileIndex: It took 89 ms to list leaf files for 1 paths.
24/06/29 11:39:40 INFO InMemoryFileIndex: It took 3 ms to list leaf files for 1 paths.
24/06/29 11:39:45 INFO FileSourceStrategy: Pushed Filters: 
24/06/29 11:39:45 INFO FileSourceStrategy: Post-Scan Filters: (length(trim(value#0, None)) > 0)
24/06/29 11:39:46 INFO CodeGenerator: Code generated in 382.260783 ms
24/06/29 11:39:46 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 200.2 KiB, free 434.2 MiB)
24/06/29 11:39:47 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 34.6 KiB, free 434.2 MiB)
24/06/29 11:39:47 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on 192.168.0.144:58763 (size: 34.6 KiB, free: 434.4 MiB)
24/06/29 11:39:47 INFO SparkContext: Created broadcast 0 from csv at NativeMethodAccessorImpl.java:0
24/06/29 11:39:47 INFO FileSourceScanExec: Planning scan with bin packing, max size: 77234390 bytes, open cost i

In [34]:
df.show(5)

24/06/29 11:40:22 INFO FileSourceStrategy: Pushed Filters: 
24/06/29 11:40:22 INFO FileSourceStrategy: Post-Scan Filters: 
24/06/29 11:40:22 INFO CodeGenerator: Code generated in 179.208574 ms
24/06/29 11:40:22 INFO MemoryStore: Block broadcast_4 stored as values in memory (estimated size 200.1 KiB, free 433.9 MiB)
24/06/29 11:40:22 INFO MemoryStore: Block broadcast_4_piece0 stored as bytes in memory (estimated size 34.6 KiB, free 433.9 MiB)
24/06/29 11:40:22 INFO BlockManagerInfo: Added broadcast_4_piece0 in memory on 192.168.0.144:58763 (size: 34.6 KiB, free: 434.3 MiB)
24/06/29 11:40:22 INFO SparkContext: Created broadcast 4 from showString at NativeMethodAccessorImpl.java:0
24/06/29 11:40:22 INFO FileSourceScanExec: Planning scan with bin packing, max size: 77234390 bytes, open cost is considered as scanning 4194304 bytes.
24/06/29 11:40:22 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
24/06/29 11:40:22 INFO DAGScheduler: Got job 2 (showString at Na

+--------------------+------+----------------+--------------+--------------------+------------------+--------------+--------------------+--------------------+--------------------+--------------------+-------------------+------------------+--------------------+-------------+-------------+--------+-----+---------+--------------------+--------------------+------+--------------+
|    Transaction_Date|Amount|Transaction_Type| Customer_Name|    Customer_Address|     Customer_City|Customer_State|    Customer_Country|             Company|           Job_Title|               Email|       Phone_Number|Credit_Card_Number|                IBAN|Currency_Code|Random_Number|Category|Group|Is_Active|        Last_Updated|         Description|Gender|Marital_Status|
+--------------------+------+----------------+--------------+--------------------+------------------+--------------+--------------------+--------------------+--------------------+--------------------+-------------------+------------------+-----

24/06/29 11:40:23 INFO CodeGenerator: Code generated in 53.918272 ms
24/06/29 11:40:23 INFO Executor: Finished task 0.0 in stage 2.0 (TID 5). 4045 bytes result sent to driver
24/06/29 11:40:23 INFO TaskSetManager: Finished task 0.0 in stage 2.0 (TID 5) in 306 ms on 192.168.0.144 (executor driver) (1/1)
24/06/29 11:40:23 INFO TaskSchedulerImpl: Removed TaskSet 2.0, whose tasks have all completed, from pool 
24/06/29 11:40:23 INFO DAGScheduler: ResultStage 2 (showString at NativeMethodAccessorImpl.java:0) finished in 0.326 s
24/06/29 11:40:23 INFO DAGScheduler: Job 2 is finished. Cancelling potential speculative or zombie tasks for this job
24/06/29 11:40:23 INFO TaskSchedulerImpl: Killing all running tasks in stage 2: Stage finished
24/06/29 11:40:23 INFO DAGScheduler: Job 2 finished: showString at NativeMethodAccessorImpl.java:0, took 0.338962 s
24/06/29 11:40:23 INFO CodeGenerator: Code generated in 58.853314 ms


In [38]:
df.printSchema()

root
 |-- Transaction_Date: timestamp (nullable = true)
 |-- Amount: double (nullable = true)
 |-- Transaction_Type: string (nullable = true)
 |-- Customer_Name: string (nullable = true)
 |-- Customer_Address: string (nullable = true)
 |-- Customer_City: string (nullable = true)
 |-- Customer_State: string (nullable = true)
 |-- Customer_Country: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Job_Title: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Phone_Number: string (nullable = true)
 |-- Credit_Card_Number: long (nullable = true)
 |-- IBAN: string (nullable = true)
 |-- Currency_Code: string (nullable = true)
 |-- Random_Number: double (nullable = true)
 |-- Category: string (nullable = true)
 |-- Group: string (nullable = true)
 |-- Is_Active: string (nullable = true)
 |-- Last_Updated: timestamp (nullable = true)
 |-- Description: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Marital_Status: string (nullable = true)

In [46]:
df.count()

1000000

In [50]:
df.columns

['Transaction_Date',
 'Amount',
 'Transaction_Type',
 'Customer_Name',
 'Customer_Address',
 'Customer_City',
 'Customer_State',
 'Customer_Country',
 'Company',
 'Job_Title',
 'Email',
 'Phone_Number',
 'Credit_Card_Number',
 'IBAN',
 'Currency_Code',
 'Random_Number',
 'Category',
 'Group',
 'Is_Active',
 'Last_Updated',
 'Description',
 'Gender',
 'Marital_Status']

In [ ]:
df.

In [40]:
## check for duplicates
# drop duplicate and count it 
df.count() - df.dropDuplicates().count()

24/06/29 11:54:58 INFO FileSourceStrategy: Pushed Filters: 
24/06/29 11:54:58 INFO FileSourceStrategy: Post-Scan Filters: 
24/06/29 11:54:58 INFO MemoryStore: Block broadcast_9 stored as values in memory (estimated size 200.1 KiB, free 433.9 MiB)
24/06/29 11:54:58 INFO MemoryStore: Block broadcast_9_piece0 stored as bytes in memory (estimated size 34.6 KiB, free 433.9 MiB)
24/06/29 11:54:58 INFO BlockManagerInfo: Added broadcast_9_piece0 in memory on 192.168.0.144:58763 (size: 34.6 KiB, free: 434.3 MiB)
24/06/29 11:54:58 INFO SparkContext: Created broadcast 9 from count at NativeMethodAccessorImpl.java:0
24/06/29 11:54:58 INFO FileSourceScanExec: Planning scan with bin packing, max size: 77234390 bytes, open cost is considered as scanning 4194304 bytes.
24/06/29 11:54:59 INFO DAGScheduler: Registering RDD 24 (count at NativeMethodAccessorImpl.java:0) as input to shuffle 1
24/06/29 11:54:59 INFO DAGScheduler: Got map stage job 5 (count at NativeMethodAccessorImpl.java:0) with 4 output p

0

In [45]:
# Sets the log level to WARN
# This will suppress INFO logs and show only WARN and ERROR log
spark.sparkContext.setLogLevel("WARN")

In [44]:
# Checking null values in all colummns and thier count 
for column in df.columns:
    print(column, 'Nulls: ', df.filter(df[column].isNull()).count())

Transaction_Date Nulls:  0


Amount Nulls:  0


Transaction_Type Nulls:  0


Customer_Name Nulls:  100425


Customer_Address Nulls:  100087


...

Is_Active Nulls:  100259


Last_Updated Nulls:  100321


Description Nulls:  100403


Gender Nulls:  99767


Marital_Status Nulls:  99904


In [52]:
df.describe().show()

24/06/29 12:10:27 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+----------------+-------------+--------------------+-------------+--------------+----------------+-------------+------------------+-------------------+--------------------+--------------------+--------------------+-------------+------------------+--------+------+---------+--------------------+------+--------------+
|summary|            Amount|Transaction_Type|Customer_Name|    Customer_Address|Customer_City|Customer_State|Customer_Country|      Company|         Job_Title|              Email|        Phone_Number|  Credit_Card_Number|                IBAN|Currency_Code|     Random_Number|Category| Group|Is_Active|         Description|Gender|Marital_Status|
+-------+------------------+----------------+-------------+--------------------+-------------+--------------+----------------+-------------+------------------+-------------------+--------------------+--------------------+--------------------+-------------+------------------+--------+------+---------+---------

In [54]:
df_clean = df.fillna({
    'Customer_Name': 'Unknown',
    'Customer_Address': 'Unknown',
    'Customer_City': 'Unknown',
    'Customer_State': 'Unknown',
    'Customer_Country': 'Unknown',
    'Company': 'Unknown',
    'Job_Title': 'Unknown',
    'Email': 'Unknown',
    'Phone_Number': 'Unknown',
    'Credit_Card_Number': 0,
    'IBAN': 'Unknown',
    'Currency_Code': 'Unknown',
    'Random_Number': 0.0,
    'Category': 'Unknown',
    'Group': 'Unknown',
    'Is_Active': 'Unknown',
    'Description': 'Unknown',
    'Gender': 'Unknown',
    'Marital_Status': ''  
})

df_clean.show(5)

+--------------------+------+----------------+--------------+--------------------+------------------+--------------+--------------------+--------------------+--------------------+--------------------+-------------------+------------------+--------------------+-------------+-------------+--------+-----+---------+--------------------+--------------------+-------+--------------+
|    Transaction_Date|Amount|Transaction_Type| Customer_Name|    Customer_Address|     Customer_City|Customer_State|    Customer_Country|             Company|           Job_Title|               Email|       Phone_Number|Credit_Card_Number|                IBAN|Currency_Code|Random_Number|Category|Group|Is_Active|        Last_Updated|         Description| Gender|Marital_Status|
+--------------------+------+----------------+--------------+--------------------+------------------+--------------+--------------------+--------------------+--------------------+--------------------+-------------------+------------------+---

In [55]:
# drop the missing values in 'Last_Updated' column
df_clean = df_clean.na.drop(subset=['Last_Updated'])

In [58]:
# check null value using SQL expression
null_check = ' OR ' .join(f'{col} is null' for col in df_clean.columns)

df_clean.select('*').filter(expr(null_check)).show()

+----------------+------+----------------+-------------+----------------+-------------+--------------+----------------+-------+---------+-----+------------+------------------+----+-------------+-------------+--------+-----+---------+------------+-----------+------+--------------+
|Transaction_Date|Amount|Transaction_Type|Customer_Name|Customer_Address|Customer_City|Customer_State|Customer_Country|Company|Job_Title|Email|Phone_Number|Credit_Card_Number|IBAN|Currency_Code|Random_Number|Category|Group|Is_Active|Last_Updated|Description|Gender|Marital_Status|
+----------------+------+----------------+-------------+----------------+-------------+--------------+----------------+-------+---------+-----+------------+------------------+----+-------------+-------------+--------+-----+---------+------------+-----------+------+--------------+
+----------------+------+----------------+-------------+----------------+-------------+--------------+----------------+-------+---------+-----+------------+-

In [59]:
df_clean.count()

899679

## Transformation 

In [63]:
# transaction table 
transaction = df_clean.select('Transaction_Date','Amount','Transaction_Type')\
                      .withColumn('transaction_id', monotonically_increasing_id())\
                      .select('transaction_id','Amount','Transaction_Type','Transaction_Date')\
                      .distinct()

In [66]:
# customer table
customer = df_clean.select('Customer_Name','Customer_Address','Customer_City','Customer_State','Customer_Country').distinct()\
                   .withColumn('customer_id', monotonically_increasing_id())\
                   .select('customer_id','Customer_Name','Customer_Address','Customer_City','Customer_State','Customer_Country')\
                   .distinct()

In [67]:
# employee table 
employee = df_clean.select('Company','Job_Title', 'Gender','Marital_Status')\
                   .withColumn('employee_id', monotonically_increasing_id())\
                   .select('employee_id','Job_Title','Company','Gender','Marital_Status')\
                   .distinct()

In [68]:
# transaction_fact table 
transaction_fact = df_clean.join(customer, ['Customer_Name','Customer_Address','Customer_City','Customer_State','Customer_Country'], 'left') \
                            .join(transaction, ['Amount','Transaction_Type','Transaction_Date'], 'left') \
                            .join(employee, ['Job_Title', 'Gender','Marital_Status'], 'left')\
                            .select('transaction_id','customer_id','employee_id','Credit_Card_Number','IBAN','Currency_Code','Random_Number','Category','Group','Is_Active','Last_Updated','Description')

transaction_fact.show(5)


+--------------+-----------+-----------+------------------+--------------------+-------------+-------------+--------+-----+---------+--------------------+--------------------+
|transaction_id|customer_id|employee_id|Credit_Card_Number|                IBAN|Currency_Code|Random_Number|Category|Group|Is_Active|        Last_Updated|         Description|
+--------------+-----------+-----------+------------------+--------------------+-------------+-------------+--------+-----+---------+--------------------+--------------------+
|             2|       7899|      16069|      675983949974|GB59QYRN446730519...|          COP|       7796.0|       C|    Z|       No|2020-01-24 01:23:...|Again line face c...|
|             2|       7899|     128537|      675983949974|GB59QYRN446730519...|          COP|       7796.0|       C|    Z|       No|2020-01-24 01:23:...|Again line face c...|
|             2|       7899|      60126|      675983949974|GB59QYRN446730519...|          COP|       7796.0|       C|   

24/06/30 01:15:35 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 3655421 ms exceeds timeout 120000 ms
24/06/30 01:15:35 WARN SparkContext: Killing executors is not supported by current scheduler.
24/06/30 01:15:41 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$

## Load to PostgreSQL